# Real-time Event Extraction from Online News and Tweets
## Proof of Concept
*Nilani Algiriyage, PhD Student, Joint Center for Disaster Research, Massey University, New Zealand.*

### [Step 1] Real-time News Scaping

In [7]:
!pip uninstall neuralcoref --yes
!pip install neuralcoref --no-binary neuralcoref

     |████████████████████████████████| 368 kB 1.0 MB/s eta 0:00:01
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
    Running setup.py install for neuralcoref ... done


In [1]:
import neuralcoref

/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [2]:
import spacy
import neuralcoref

In [1]:
import spacy
nlp = spacy.load('en')

# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

# # You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
# doc = nlp(u'My sister has a dog. She loves him.')

# doc._.has_coref
# doc._.coref_clusters

/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/rangika/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:
doc = nlp(u'My sister has a dog. She loves him.')

In [2]:
#import libraries
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from newspaper import Article
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim
from itertools import chain
import operator
import string
import warnings 
import folium
from collections import Counter
import spacy 
from spacy.lang.en import English # updated
from spacy.matcher import Matcher
pd.options.mode.chained_assignment = None  # default='warn'
#pd.set_option('display.max_colwidth', 0)
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

#### [1.1] Herald News

In [2]:
df_herald = pd.DataFrame([['One dead after two motorbikes crash in Bay of Plenty',
 'Raglan shooting: Community detention for man who bought gun for accused murderer',
 'One person seriously injured and in hospital after Hamilton house fire',
 'Multiple injuries in State Highway 29 crash near Tauriko, Tauranga'],['https://www.nzherald.co.nz/nz/news/article.cfm?c_id=1&objectid=12352774','https://www.nzherald.co.nz/nz/news/article.cfm?c_id=1&objectid=12352605','https://www.nzherald.co.nz/nz/news/article.cfm?c_id=1&objectid=12352756','https://www.nzherald.co.nz/nz/news/article.cfm?c_id=1&objectid=12352739'],['2020-08-01 08:38:56+12:00','2020-08-01 08:45:12+12:00','2020-08-01 08:55:23+12:00','2020-08-01 08:57:56+12:00']]).T
df_herald.columns = ['Title', 'Link','Date']
df_herald

,Title,Link,Date
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00
1,Raglan shooting: Community detention for man w...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:45:12+12:00
2,One person seriously injured and in hospital a...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:55:23+12:00
3,Multiple injuries in State Highway 29 crash ne...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:57:56+12:00


#### [1.2] RNZ News

In [3]:
df_rnz = pd.DataFrame([['Motorcycles crash leaves one dead near Whakatāne',
 'One person seriously injured in Hamilton house fire'],['https://www.rnz.co.nz/news/national/422481/motorcycles-crash-leaves-one-dead-near-whakatane','https://www.rnz.co.nz/news/national/422458/one-person-seriously-injured-in-hamilton-house-fire'],['2020-08-01 08:30:45+12:00','2020-08-01 08:45:56+12:00']]).T
df_rnz.columns = ['Title', 'Link','Date']
df_rnz

,Title,Link,Date
0,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00
1,One person seriously injured in Hamilton house...,https://www.rnz.co.nz/news/national/422458/one...,2020-08-01 08:45:56+12:00


### [Step 2] Real-time Tweet Extraction

In [4]:
df_tweet = pd.DataFrame([["burn from Hamilton but it's me setting fire to my sims' house with them locked in it"],['2020-08-01 08:38:56+12:00']]).T
df_tweet.columns = ['Title','Date']
df_tweet

,Title,Date
0,burn from Hamilton but it's me setting fire to...,2020-08-01 08:38:56+12:00


In [5]:
list1 = df_herald.Title.to_list()
list2 = df_rnz.Title.to_list()
list3 = df_tweet.Title.to_list()

finalList = list1 + list2 + list3
finalList

['One dead after two motorbikes crash in Bay of Plenty',
 'Raglan shooting: Community detention for man who bought gun for accused murderer',
 'One person seriously injured and in hospital after Hamilton house fire',
 'Multiple injuries in State Highway 29 crash near Tauriko, Tauranga',
 'Motorcycles crash leaves one dead near Whakatāne',
 'One person seriously injured in Hamilton house fire',
 "burn from Hamilton but it's me setting fire to my sims' house with them locked in it"]

### [3] Cluster News Titles and Tweets

In [6]:
nlp = spacy.load('en_core_web_lg')
sent_vecs = {}
docs = []
for title in finalList:
    doc = nlp(title)
    docs.append(doc)
    sent_vecs.update({title:doc.vector})
    
sentences = list(sent_vecs.keys())
vectors = list(sent_vecs.values())

vectors

[array([ 2.03693584e-01,  9.59407985e-02, -1.36530399e-01, -1.14597306e-01,
         1.77369907e-01, -1.09860219e-01, -8.65866989e-02, -8.98338407e-02,
         1.53838933e-01,  2.20342398e+00,  2.31505074e-02, -5.14205992e-02,
         3.21482047e-02,  4.59412113e-02, -3.49897176e-01, -1.33033603e-01,
        -7.08606094e-02,  1.00682104e+00,  4.19289991e-02, -7.72764012e-02,
        -1.68773293e-01,  4.60688993e-02,  1.74670704e-02, -6.49686009e-02,
         1.18285105e-01, -1.19821407e-01, -1.51784211e-01,  3.07731479e-02,
         2.05921322e-01,  2.39058286e-02,  3.24502122e-04,  1.60527825e-02,
        -6.77995980e-02,  1.45091504e-01,  7.32080117e-02, -4.34606001e-02,
         3.93132977e-02,  1.01313397e-01, -1.29441336e-01,  6.01693988e-02,
         4.91640251e-03, -6.90798908e-02, -1.70688070e-02,  1.77018978e-02,
         5.46867959e-02,  1.07588388e-01, -6.62437081e-02,  6.57183081e-02,
         8.24853033e-02,  2.55928393e-02, -4.64865938e-02, -5.19636050e-02,
         5.1

In [7]:
x = np.array(vectors)
n_classes = {}

In [8]:
for i in np.arange(0.001, 1, 0.002):
    dbscan = DBSCAN(eps=i, min_samples=2, metric='cosine').fit(x)
    n_classes.update({i:len(pd.Series(dbscan.labels_).value_counts())})
    
dbscan = DBSCAN(eps=0.15, min_samples=2, metric='cosine').fit(x)

In [9]:
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

Estimated number of clusters: 2
Estimated number of noise points: 2


In [10]:
labels

array([ 0, -1,  1, -1,  0,  1,  1])

In [11]:
results = pd.DataFrame({'label':dbscan.labels_, 'sent':sentences})
example_result = results[results.label == 0].sent.tolist()
example_result

['One dead after two motorbikes crash in Bay of Plenty',
 'Motorcycles crash leaves one dead near Whakatāne']

In [12]:
df4 = df_herald[df_herald['Title'].isin(example_result)]
df5 = df_rnz[df_rnz['Title'].isin(example_result)]
#df6 = df_tweet[df_tweet['Title'].isin(example_result)]

In [13]:
bigdata = pd.concat([df4,df5], ignore_index=True)
bigdata

,Title,Link,Date
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00


### [4] Extract candidates for When, Where, What, from Newa Headlines and Tweets 

#### [4.1] When? -from Title

In [14]:
nlp = spacy.load('en_core_web_lg')
bigdata['When_Title'] = bigdata['Title'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if ent.label_== "DATE" or ent.label_=='TIME']))
bigdata['When_Title']

0    
1    
Name: When_Title, dtype: object

#### [4.2] Where? -from Title

In [15]:
#nlp = spacy.load('en_core_web_lg')

bigdata['Where_Title'] = bigdata['Title'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if (ent.label_=='GPE' or ent.label_=='LOC' or ent.label_=='FAC')]))
bigdata['Where_Title']

0    Bay of Plenty
1                 
Name: Where_Title, dtype: object

#### [4.3] What? -from Title

In [16]:
nlp = spacy.load("en_core_web_sm") 
def dep_subtree(token, dep):
  #print(token, dep)
  deps =[child.dep_ for child in token.children]
  
  child=next(filter(lambda c: c.dep_==dep, token.children), None)
  #print(child)
  if child != None:
    return " ".join([c.text for c in child.subtree])
  else:
    return ""



def extract_events_spacy(line):
    #line=p.sub('', line)
    events = []
    doc = nlp(line)
    for tok in doc:
        current = tok
        if current.dep_ == "ROOT":
            current = current.head
            #print(current.text)
            desc = " ".join(filter(None,[
                                 current.text,
                                 dep_subtree(current,"dobj")]))

    events = events + [(desc)]
    return events

In [17]:
bigdata['What_Title'] = bigdata['Title'].apply(extract_events_spacy)
bigdata.head()

,Title,Link,Date,When_Title,Where_Title,What_Title
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,[dead]
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,[leaves one dead]


In [18]:
bigdata['What_Title'] = bigdata.What_Title.astype(str).str.replace('\[|\]|\'', '')
bigdata['What_Title']

0               dead
1    leaves one dead
Name: What_Title, dtype: object

### [5] Scrape News Body Text

In [19]:
bigdata = bigdata.dropna()  
eventURLList = bigdata['Link'].tolist()

#PubDate = []
#NewsTitle = []
NewsData = []
for url in eventURLList:
    article = Article(url)
    article.download()
    article.parse()
    #PubDate.append(article.publish_date)
    NewsData.append(article.text)
    #NewsTitle.append(article.title)
    


bigdata['Description'] = pd.Series(NewsData, index=bigdata.index)
bigdata.head()
#eventURLList

,Title,Link,Date,When_Title,Where_Title,What_Title,Description
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,leaves one dead,One person has died and another has minor inju...


In [20]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [21]:
#stop_words = set(stopwords.words('english'))
def  text_cleaner2(text):
    newString = text
    
    #newString = newString.replace('/', '')
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub('\""(.+?)\""', '', newString)#remove double quotes
    newString = re.sub('Advertisement', '', newString)# Remove 'Headline:' part
    newString = re.sub('Photo', '', newString)
    newString = re.sub('''Video will play in Play now Don't auto play''', '', newString)
    newString = re.sub('Never auto play', '', newString)
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = (re.sub(r'\b[A-Z]+\b', '', newString))#remove capital letters
    newString = re.sub('"','', newString)
    newString = re.sub(r"http\S+", "", newString)#remove twitter links
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub(',','', newString)#remove commas
    #newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    #newString = re.sub(r"'s\b","",newString)
    #newString = re.sub(r'[^\w\s]','',newString)#remove punctuation
    return newString

bigdata['Tidy_News'] = np.vectorize(text_cleaner2)(bigdata['Description'])
bigdata.head()

,Title,Link,Date,When_Title,Where_Title,What_Title,Description,Tidy_News
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,leaves one dead,One person has died and another has minor inju...,One person has died and another has minor inju...


#### [5.1] Extract first N sentences using the Inverted Pyramid Structure

In [22]:
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated


def split_sentences(document):
    SentencesList = []
    doc = nlp(document)
    sentences = [sent.string.strip() for sent in doc.sents][:6]
    
    SentencesList.append(sentences)
    #print(len(sentences))
    #print(SentencesList)
    return SentencesList



#split_sentences(string1)
#df2['first_sentences'] = np.vectorize(split_sentences)(df2['Description'])

bigdata['first_sentences'] = bigdata['Tidy_News'].apply(split_sentences)
bigdata.head()

,Title,Link,Date,When_Title,Where_Title,What_Title,Description,Tidy_News,first_sentences
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[[One person has died after two motorbikes cra...
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,leaves one dead,One person has died and another has minor inju...,One person has died and another has minor inju...,[[One person has died and another has minor in...


In [23]:
#bigdata['liststring'] = bigdata['first_sentences'].apply(lambda x: ','.join(map(str, x)))
bigdata['liststring'] = bigdata.first_sentences.astype(str).str.replace('\[|\]|\'', '')
bigdata.head(1)

,Title,Link,Date,When_Title,Where_Title,What_Title,Description,Tidy_News,first_sentences,liststring
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[[One person has died after two motorbikes cra...,One person has died after two motorbikes crash...


#### [5.2] When? -from Body

In [24]:
#nlp = spacy.load('en_core_web_lg')

bigdata['When_Body'] = bigdata['liststring'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if ent.label_== "DATE" or ent.label_=='TIME']))
bigdata['When_Body']

0    
1    
Name: When_Body, dtype: object

In [25]:
def time_parse(line):
    timestrings = []
    r = re.compile(r'(\d*[:.]\d{2}\s?(?:AM|PM|am|pm))')
    x = r.findall(line)
    timestrings.append(x)
    return timestrings

In [26]:
bigdata['When_Body_Regex'] = bigdata['liststring'].apply(lambda row: time_parse(row))
bigdata['When_Body_Regex'] = bigdata.When_Body_Regex.astype(str).str.replace('\[|\]|\'', '')
bigdata['When_Body_Regex']

0    3.15pm
1    3.15pm
Name: When_Body_Regex, dtype: object

#### [5.3] Where? -from Body

In [27]:
nlp = spacy.load('en_core_web_lg')
bigdata['Where_Body'] = bigdata['liststring'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if (ent.label_=='GPE' or ent.label_=='LOC' or ent.label_=='FAC')]))
bigdata['Where_Body']

0    the Bay of Plenty,Bell Rd,Nukuhou
1        Nukuhou,Whakatāne,Eastern Bay
Name: Where_Body, dtype: object

#### [5.4] What? -from Body

In [28]:
nlp = spacy.load("en_core_web_sm") 
def dep_subtree(token, dep):
  #print(token, dep)
  deps =[child.dep_ for child in token.children]
  
  child=next(filter(lambda c: c.dep_==dep, token.children), None)
  #print(child)
  if child != None:
    return " ".join([c.text for c in child.subtree])
  else:
    return ""



def extract_events_spacy2(line):
    #line=p.sub('', line)
    events = []
    
    for l in line.split('.'):
#         l = re.sub('said', '', l)
#         l = re.sub('asked', '', l)
#         l = re.sub('were', '', l)
        #print(l)
        doc = nlp(l)
        for tok in doc:
            current = tok
            if current.dep_ == "ROOT":
                current = current.head
                #print(current.text)
                desc = " ".join(filter(None,[
                                 current.text,
                                 dep_subtree(current,"dobj")]))

        events = events + [(desc)]
        events[:] = [x for x in events if "asked" not in x]
        events[:] = [x for x in events if "said" not in x]
    return events

In [29]:
bigdata['What_Body'] = bigdata['liststring'].apply(extract_events_spacy2)
bigdata.head()

,Title,Link,Date,When_Title,Where_Title,What_Title,Description,Tidy_News,first_sentences,liststring,When_Body,When_Body_Regex,Where_Body,What_Body
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[[One person has died after two motorbikes cra...,One person has died after two motorbikes crash...,,3.15pm,"the Bay of Plenty,Bell Rd,Nukuhou","[died, occurred, suffered minor injuries]"
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,leaves one dead,One person has died and another has minor inju...,One person has died and another has minor inju...,[[One person has died and another has minor in...,One person has died and another has minor inju...,,3.15pm,"Nukuhou,Whakatāne,Eastern Bay","[died, involved two motorcycles, 15pm, attendi..."


In [30]:
bigdata['What_Body'] = bigdata.What_Body.astype(str).str.replace('\[|\]|\'', '')
bigdata.head()

,Title,Link,Date,When_Title,Where_Title,What_Title,Description,Tidy_News,first_sentences,liststring,When_Body,When_Body_Regex,Where_Body,What_Body
0,One dead after two motorbikes crash in Bay of ...,https://www.nzherald.co.nz/nz/news/article.cfm...,2020-08-01 08:38:56+12:00,,Bay of Plenty,dead,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[[One person has died after two motorbikes cra...,One person has died after two motorbikes crash...,,3.15pm,"the Bay of Plenty,Bell Rd,Nukuhou","died, occurred, suffered minor injuries"
1,Motorcycles crash leaves one dead near Whakatāne,https://www.rnz.co.nz/news/national/422481/mot...,2020-08-01 08:30:45+12:00,,,leaves one dead,One person has died and another has minor inju...,One person has died and another has minor inju...,[[One person has died and another has minor in...,One person has died and another has minor inju...,,3.15pm,"Nukuhou,Whakatāne,Eastern Bay","died, involved two motorcycles, 15pm, attendin..."


In [31]:
bigdata['liststring'].tolist()

['One person has died after two motorbikes crashed in the Bay of Plenty., The crash occurred at 3.15pm on Bell Rd in Nukuhou south of Whakatāne police said., Another person suffered minor injuries in the crash., WorkSafe had been advised and the Serious Crash Unit was in attendance police said.',
 'One person has died and another has minor injuries following a serious crash in Nukuhou near Whakatāne in Eastern Bay of Plenty., The crash involved two motorcycles and was reported around 3.15pm., The Serious Crash Unit and WorkSafe are attending the scene.']

### [6] Candidate Scoring

#### [6.1] Where

In [32]:
WT1 = bigdata['Where_Title'].tolist()
WT2 = bigdata['Where_Body'].tolist()
WT1, WT2

(['Bay of Plenty', ''],
 ['the Bay of Plenty,Bell Rd,Nukuhou', 'Nukuhou,Whakatāne,Eastern Bay'])

In [36]:
WT = WT1 + WT2 
WT

words = [w.replace('the ', '') for w in WT]
words = [w.replace('Eastern Bay', 'Eastern Bay of Plenty') for w in words]
words = [w.replace('Eastern', '') for w in words]
words = [w.replace('house', '') for w in words]


str_list = list(filter(None, words))
str_list

['Bay of Plenty',
 'Bay of Plenty,Bell Rd,Nukuhou',
 'Nukuhou,Whakatāne, Bay of Plenty']

In [37]:
str_list2 = ",".join(str_list).split(',')
str_list2 = [x.strip(' ') for x in str_list2] #Remove space before ' Bay of Plenty
str_list2

['Bay of Plenty',
 'Bay of Plenty',
 'Bell Rd',
 'Nukuhou',
 'Nukuhou',
 'Whakatāne',
 'Bay of Plenty']

In [38]:
counts = Counter(str_list2)
counts_where = dict(counts)
print(counts_where)

{'Bay of Plenty': 3, 'Bell Rd': 1, 'Nukuhou': 2, 'Whakatāne': 1}


In [39]:
# d = {'frequency': 0.9, 'position': 0.8, 'relatedness': 0.7}

# frequecy_weight = pd.DataFrame([d], columns=d.keys())

# frequecy_weight

# frequecy_weight = pd.DataFrame([['frequency','position','relatedness'], [0.9, 0.8, 0.7]]).T
# frequecy_weight.columns = ['description','weight']
# frequecy_weight

In [40]:
df_locations = pd.DataFrame(list(counts_where.items()), columns=['locations', 'frequency'])
df_locations

,locations,frequency
0,Bay of Plenty,3
1,Bell Rd,1
2,Nukuhou,2
3,Whakatāne,1


In [41]:
loc_list = df_locations.locations.tolist()
loc_list

['Bay of Plenty', 'Bell Rd', 'Nukuhou', 'Whakatāne']

### Position Score

In [42]:
titletext = bigdata[["Title"]]
titletext

,Title
0,One dead after two motorbikes crash in Bay of ...
1,Motorcycles crash leaves one dead near Whakatāne


In [43]:
titletext['locs_in_title'] = titletext.Title.str.findall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
titletext

,Title,locs_in_title
0,One dead after two motorbikes crash in Bay of ...,[Bay of Plenty]
1,Motorcycles crash leaves one dead near Whakatāne,[Whakatāne]


In [44]:
titletext['locs_in_title'] = titletext.locs_in_title.astype(str).str.replace('\[|\]|\'', '')
titletext

,Title,locs_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne


In [45]:
titletext['locs_in_title'].replace('', np.nan, inplace=True)
titletext

,Title,locs_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne


In [46]:
titletext = titletext.dropna(subset=['locs_in_title'])
titletext
# def y(row):
#     if row['locs_in_title'] == '':
#         val = 0.0
#     else:
#         val = 1.0
#     return val

,Title,locs_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne


In [47]:
#df[df['column_name'] == '']
#titletext['title_score'] =  titletext.apply(y, axis=1)
titletext['title_score'] = 1.0
titletext

,Title,locs_in_title,title_score
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty,1.0
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne,1.0


In [48]:
bodytext = bigdata[["liststring"]]

bodytext["Shape"]= bodytext["liststring"].str.split("., ")

bodytext = (bodytext.explode("Shape").reset_index(drop=True))

bodytext

,liststring,Shape
0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
4,One person has died and another has minor inju...,One person has died and another has minor inju...
5,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
6,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [49]:
grouped = bodytext.groupby('liststring', as_index=False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
result = grouped_index.drop('level_0',axis = 1).set_index('level_1')
result

,liststring,Shape
level_1,,
0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
0,One person has died and another has minor inju...,One person has died and another has minor inju...
1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [50]:
result = result.reset_index()
result

,level_1,liststring,Shape
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [51]:
import re
# result['location_mapped_title'] = (result.Shape.str
#                              .findall('|'.join(loc_list))
#                              .str[0])

#x = result.Shape.str.extractall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result['locs_in_lines'] = result.Shape.str.findall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result
#loc_list = df_locations.location.tolist()
#[i.split('.,')[0] for i in sentence_list] 

#[item.upper() for item in mylis]

,level_1,liststring,Shape,locs_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[Bay of Plenty]
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"[Bell Rd, Nukuhou, Whakatāne]"
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,[]
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,[]
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,"[Nukuhou, Whakatāne, Bay of Plenty]"
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,[]
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,[]


In [52]:
# df_calc = result[['level_1', 'locs_in_lines']]
# df_calc

In [53]:
result['locs_in_lines'] = result.locs_in_lines.astype(str).str.replace('\[|\]|\'', '')

result

,level_1,liststring,Shape,locs_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,Bay of Plenty
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"Bell Rd, Nukuhou, Whakatāne"
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,"Nukuhou, Whakatāne, Bay of Plenty"
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,


In [54]:
result['locs_in_lines'].replace("", np.nan, inplace=True)
result = result.dropna(subset=['locs_in_lines'])
result

,level_1,liststring,Shape,locs_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,Bay of Plenty
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"Bell Rd, Nukuhou, Whakatāne"
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,"Nukuhou, Whakatāne, Bay of Plenty"


In [55]:
def f(row):
    if row['level_1'] == 0:
        val = 1
    elif row['level_1'] == 1:
        val = 0.5
    elif row['level_1'] == 2:
        val = 0.25
    elif row['level_1'] == 3:
        val = 0.125
    elif row['level_1'] == 4:
        val = 0.0625
    else:
        val = 0.03125
    return val

In [56]:
result['body_score'] = result.apply(f, axis=1)
result

,level_1,liststring,Shape,locs_in_lines,body_score
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,Bay of Plenty,1.0
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"Bell Rd, Nukuhou, Whakatāne",0.5
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,"Nukuhou, Whakatāne, Bay of Plenty",1.0


In [57]:
#df_calc = result[['locs_in_lines', 'pos_score']]
result["locations"]=result["locs_in_lines"].str.split(", ")
result = result.explode("locations")

#df_calc = df_calc.groupby('splited')['C'].sum().reset_index() 
print(result)

   level_1                                         liststring  \
0        0  One person has died after two motorbikes crash...   
1        1  One person has died after two motorbikes crash...   
1        1  One person has died after two motorbikes crash...   
1        1  One person has died after two motorbikes crash...   
4        0  One person has died and another has minor inju...   
4        0  One person has died and another has minor inju...   
4        0  One person has died and another has minor inju...   

                                               Shape  \
0  One person has died after two motorbikes crash...   
1  The crash occurred at 3.15pm on Bell Rd in Nuk...   
1  The crash occurred at 3.15pm on Bell Rd in Nuk...   
1  The crash occurred at 3.15pm on Bell Rd in Nuk...   
4  One person has died and another has minor inju...   
4  One person has died and another has minor inju...   
4  One person has died and another has minor inju...   

                       locs_in

In [58]:
df_bodytext_score = result.groupby('locations')['body_score'].sum().reset_index() 
df_bodytext_score

,locations,body_score
0,Bay of Plenty,2.0
1,Bell Rd,0.5
2,Nukuhou,1.5
3,Whakatāne,1.5


In [59]:
df_title_score = titletext[['locs_in_title', 'title_score']]
df_title_score['locations'] = df_title_score['locs_in_title']
df_title_score

,locs_in_title,title_score,locations
0,Bay of Plenty,1.0,Bay of Plenty
1,Whakatāne,1.0,Whakatāne


In [60]:
df_position_score = pd.merge(df_bodytext_score, df_title_score, on=['locations'],how='outer')
df_position_score

,locations,body_score,locs_in_title,title_score
0,Bay of Plenty,2.0,Bay of Plenty,1.0
1,Bell Rd,0.5,NaN,NaN
2,Nukuhou,1.5,NaN,NaN
3,Whakatāne,1.5,Whakatāne,1.0


In [61]:
df_position_score['full_position_score'] = df_position_score['body_score'] + df_position_score['title_score']
df_position_score

,locations,body_score,locs_in_title,title_score,full_position_score
0,Bay of Plenty,2.0,Bay of Plenty,1.0,3.0
1,Bell Rd,0.5,NaN,NaN,NaN
2,Nukuhou,1.5,NaN,NaN,NaN
3,Whakatāne,1.5,Whakatāne,1.0,2.5


In [62]:
df_position_score.full_position_score.fillna(df_position_score.body_score, inplace=True)
df_position_score

,locations,body_score,locs_in_title,title_score,full_position_score
0,Bay of Plenty,2.0,Bay of Plenty,1.0,3.0
1,Bell Rd,0.5,NaN,NaN,0.5
2,Nukuhou,1.5,NaN,NaN,1.5
3,Whakatāne,1.5,Whakatāne,1.0,2.5


In [63]:
#df_locations['splited'] = df_locations['location']
df_locations

,locations,frequency
0,Bay of Plenty,3
1,Bell Rd,1
2,Nukuhou,2
3,Whakatāne,1


In [64]:
df_position_and_frequency = pd.merge(df_locations, df_position_score, on=['locations'],how='outer')
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0
1,Bell Rd,1,0.5,NaN,NaN,0.5
2,Nukuhou,2,1.5,NaN,NaN,1.5
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5


In [65]:
df_position_and_frequency['position_and_frequency_score'] = df_position_and_frequency['full_position_score'] + df_position_and_frequency['frequency']
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0,6.0
1,Bell Rd,1,0.5,NaN,NaN,0.5,1.5
2,Nukuhou,2,1.5,NaN,NaN,1.5,3.5
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5,3.5


### Relatedness Score

In [66]:
from geopy.geocoders import Nominatim

def geo_code(line):
    x = []
    y = []

    line2 = 'New Zealand'
    n = line + ' + ' + line2
    r = '"' + n + '"'
    


    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(r)
    x.append(location.latitude)
    y.append(location.longitude)
    return x,y

In [67]:
#listofloc = list(set(str_list2))
#cordinates = list(map(geo_code, listofloc))
loc_list
# cordinates = [geo_code(item) for item in listofloc]
# #cordinates
# list_of_coords = [(x[0][0], x[1][0]) for x in cordinates]
# list_of_coords

['Bay of Plenty', 'Bell Rd', 'Nukuhou', 'Whakatāne']

In [68]:
cordinates = [geo_code(item) for item in loc_list]
cordinates

[([-37.9503756], [176.93828736155422]),
 ([-43.6131375], [172.3051281]),
 ([-38.082811], [177.1341941]),
 ([-37.9519223], [176.9945977])]

In [69]:
list_of_coords = [(x[0][0], x[1][0]) for x in cordinates]
list_of_coords

[(-37.9503756, 176.93828736155422),
 (-43.6131375, 172.3051281),
 (-38.082811, 177.1341941),
 (-37.9519223, 176.9945977)]

In [70]:
from math import hypot
from geopy.distance import geodesic

#https://stackoverflow.com/questions/23086078/how-to-iterate-through-a-list-of-coordinates-and-calculate-distance-between-them
def distance(p1,p2):
    x = geodesic(p1, p2).miles
    """Euclidean distance between two points."""
#     x1,y1 = p1
#     x2,y2 = p2

    return x

from itertools import combinations
#list_of_coords = [(-37.9503756, 176.93828736155422), (-43.6131375, 172.3051281), (-38.1272259, 177.1307049)]



distancecombi = [distance(*combo) for combo in combinations(list_of_coords,2)]
distancecombi

[459.9687201814059,
 14.060479803232681,
 3.0771117214399695,
 457.69033046543694,
 461.43937616613573,
 11.811492181575764]

In [71]:
import itertools

locationombi = list(itertools.combinations(loc_list, 2))# cordinates = list(map(geo_code, listofloc))
locationombi

[('Bay of Plenty', 'Bell Rd'),
 ('Bay of Plenty', 'Nukuhou'),
 ('Bay of Plenty', 'Whakatāne'),
 ('Bell Rd', 'Nukuhou'),
 ('Bell Rd', 'Whakatāne'),
 ('Nukuhou', 'Whakatāne')]

In [72]:
import pandas as pd
import numpy as np

df_loc = pd.DataFrame(
    {'locpairs': locationombi,
     'dispairs': distancecombi
    })
df_loc

,locpairs,dispairs
0,"(Bay of Plenty, Bell Rd)",459.968720
1,"(Bay of Plenty, Nukuhou)",14.060480
2,"(Bay of Plenty, Whakatāne)",3.077112
3,"(Bell Rd, Nukuhou)",457.690330
4,"(Bell Rd, Whakatāne)",461.439376
5,"(Nukuhou, Whakatāne)",11.811492


In [73]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn import preprocessing
# scaler = preprocessing.MinMaxScaler()
# df_loc['locnormalized'] = scaler.fit_transform(df_loc[['dispairs']])
# df_loc

In [74]:
df_loc['locpairs'] = df_loc.locpairs.astype(str).str.replace('\(|\)|\'', '')
df_loc

,locpairs,dispairs
0,"Bay of Plenty, Bell Rd",459.968720
1,"Bay of Plenty, Nukuhou",14.060480
2,"Bay of Plenty, Whakatāne",3.077112
3,"Bell Rd, Nukuhou",457.690330
4,"Bell Rd, Whakatāne",461.439376
5,"Nukuhou, Whakatāne",11.811492


In [75]:
def x(row):
    if row['dispairs'] <= 10:
        val = 1
    elif row['dispairs'] <= 20:
        val = 0.5
    elif row['dispairs'] <= 30:
        val = 0.25
    elif row['dispairs'] <= 40:
        val = 0.125
    elif row['dispairs'] <= 50:
        val = 0.0625
    else:
        val = 0.03125
    return val

In [76]:
df_loc['loc_score'] = df_loc.apply(x, axis=1)
df_loc

,locpairs,dispairs,loc_score
0,"Bay of Plenty, Bell Rd",459.968720,0.03125
1,"Bay of Plenty, Nukuhou",14.060480,0.50000
2,"Bay of Plenty, Whakatāne",3.077112,1.00000
3,"Bell Rd, Nukuhou",457.690330,0.03125
4,"Bell Rd, Whakatāne",461.439376,0.03125
5,"Nukuhou, Whakatāne",11.811492,0.50000


In [77]:
df_relatedness = df_loc.loc[df_loc['loc_score'] == df_loc.loc_score.max()]
df_relatedness

,locpairs,dispairs,loc_score
2,"Bay of Plenty, Whakatāne",3.077112,1.0


In [78]:
df_relatedness["locations"]=df_relatedness["locpairs"].str.split(", ")
df_relatedness = df_relatedness.explode("locations")
df_relatedness

,locpairs,dispairs,loc_score,locations
2,"Bay of Plenty, Whakatāne",3.077112,1.0,Bay of Plenty
2,"Bay of Plenty, Whakatāne",3.077112,1.0,Whakatāne


In [79]:
#df_all_score2 = pd.merge(df_all_score, df_relatedness, on=['splited'],how='outer')
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0,6.0
1,Bell Rd,1,0.5,NaN,NaN,0.5,1.5
2,Nukuhou,2,1.5,NaN,NaN,1.5,3.5
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5,3.5


In [80]:
# df_relatedness2  = df_relatedness[['splited','loc_score']]
# df_relatedness2
# df_loc.loc[df_loc['locnormalized'] == df_loc.locnormalized.min(), 'locpairs'].tolist()
#loc_list = df_loc.loc[df_loc['dispairs'] <= 20, 'locpairs'].tolist()
#df_locations[df_locations['location'].isin(loc_list)]
#loc_list

In [81]:
df_relatedness.drop_duplicates(subset=['locations'], inplace=True)
df_relatedness

,locpairs,dispairs,loc_score,locations
2,"Bay of Plenty, Whakatāne",3.077112,1.0,Bay of Plenty
2,"Bay of Plenty, Whakatāne",3.077112,1.0,Whakatāne


In [82]:
df_relatedness2  = df_relatedness[['locations','loc_score']]
df_relatedness2

,locations,loc_score
2,Bay of Plenty,1.0
2,Whakatāne,1.0


In [83]:
df_all_location_score = pd.merge(df_position_and_frequency, df_relatedness2, on=['locations'],how='outer')
df_all_location_score

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score,loc_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0,6.0,1.0
1,Bell Rd,1,0.5,NaN,NaN,0.5,1.5,NaN
2,Nukuhou,2,1.5,NaN,NaN,1.5,3.5,NaN
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5,3.5,1.0


In [84]:
df_all_location_score['full_location_score'] = df_all_location_score['position_and_frequency_score'] + df_all_location_score['loc_score']
df_all_location_score

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score,loc_score,full_location_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0,6.0,1.0,7.0
1,Bell Rd,1,0.5,NaN,NaN,0.5,1.5,NaN,NaN
2,Nukuhou,2,1.5,NaN,NaN,1.5,3.5,NaN,NaN
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5,3.5,1.0,4.5


In [85]:
df_all_location_score.full_location_score.fillna(df_all_location_score.position_and_frequency_score, inplace=True)
df_all_location_score

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score,loc_score,full_location_score
0,Bay of Plenty,3,2.0,Bay of Plenty,1.0,3.0,6.0,1.0,7.0
1,Bell Rd,1,0.5,NaN,NaN,0.5,1.5,NaN,1.5
2,Nukuhou,2,1.5,NaN,NaN,1.5,3.5,NaN,3.5
3,Whakatāne,1,1.5,Whakatāne,1.0,2.5,3.5,1.0,4.5


In [86]:
#loc = max(counts_where.items(), key=operator.itemgetter(1))[0] #get the maximum value in the dict
#loc = df_all_score2['location'].values[0]
loc = df_all_location_score.loc[df_all_location_score['full_location_score'].idxmax()].values[0]
#df_all_score2.full_score.max()
print("=============================")
print("Where? : ", loc)
print("=============================")

Where? :  Bay of Plenty


In [87]:
geocodes = geo_code(loc)
geocodes

([-37.9503756], [176.93828736155422])

In [88]:
listoflist = [list(row) for row in geocodes]
flatlist = list(chain.from_iterable(listoflist))

In [89]:
locationlist = [flatlist*1]

map = folium.Map(zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map)
map

#### [6.2] When?

In [90]:
When_Title = bigdata['When_Title'].tolist()
When_Body = bigdata['When_Body'].tolist()
When_Body_Regex = bigdata['When_Body_Regex'].tolist()

When_Title, When_Body, When_Body_Regex

(['', ''], ['', ''], ['3.15pm', '3.15pm'])

In [91]:
All_When = When_Title + When_Body + When_Body_Regex
All_When

# words = [w.replace('the', '') for w in WT]
# words = [w.replace('Eastern Bay', 'Eastern Bay of Plenty') for w in words]
# words = [w.replace('Eastern', '') for w in words]

All_When_List = list(filter(None, All_When))
All_When_List

['3.15pm', '3.15pm']

In [92]:
counts = Counter(All_When_List)
counts_when = dict(counts)
print(counts_when)

{'3.15pm': 2}


In [93]:
df_when_frequency = pd.DataFrame(list(counts_when.items()), columns=['times', 'frequency'])
df_when_frequency

,times,frequency
0,3.15pm,2


In [94]:
time_list = df_when_frequency.times.tolist()
time_list

['3.15pm']

#### when? - position score

In [95]:
titletext['times_in_title'] = titletext.Title.str.findall(f'({"|".join(time_list)})', flags=re.IGNORECASE)
titletext

,Title,locs_in_title,title_score,times_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty,1.0,[]
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne,1.0,[]


In [96]:
titletext['times_in_title'] = titletext.times_in_title.astype(str).str.replace('\[|\]|\'', '')
titletext

,Title,locs_in_title,title_score,times_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty,1.0,
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne,1.0,


In [97]:
titletext['times_in_title'].replace('', np.nan, inplace=True)
titletext

,Title,locs_in_title,title_score,times_in_title
0,One dead after two motorbikes crash in Bay of ...,Bay of Plenty,1.0,NaN
1,Motorcycles crash leaves one dead near Whakatāne,Whakatāne,1.0,NaN


In [98]:
titletext = titletext.dropna(subset=['times_in_title'])
titletext

,Title,locs_in_title,title_score,times_in_title


In [99]:
if not titletext.empty:
    titletext['when_title_score'] = 1.0
titletext

,Title,locs_in_title,title_score,times_in_title


In [100]:
grouped = bodytext.groupby('liststring', as_index=False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
result = grouped_index.drop('level_0',axis = 1).set_index('level_1')
result

,liststring,Shape
level_1,,
0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
0,One person has died and another has minor inju...,One person has died and another has minor inju...
1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [101]:
result = result.reset_index()
result

,level_1,liststring,Shape
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [102]:
import re
# result['location_mapped_title'] = (result.Shape.str
#                              .findall('|'.join(loc_list))
#                              .str[0])

#x = result.Shape.str.extractall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result['times_in_lines'] = result.Shape.str.findall(f'({"|".join(time_list)})', flags=re.IGNORECASE)
result

,level_1,liststring,Shape,times_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[]
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,[3.15pm]
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,[]
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,[]
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,[]
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,[3.15pm]
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,[]


In [103]:
result['times_in_lines'] = result.times_in_lines.astype(str).str.replace('\[|\]|\'', '')

result

,level_1,liststring,Shape,times_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,3.15pm
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,3.15pm
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,


In [104]:
result['times_in_lines'].replace("", np.nan, inplace=True)
result = result.dropna(subset=['times_in_lines'])
result

,level_1,liststring,Shape,times_in_lines
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,3.15pm
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,3.15pm


In [105]:
result['when_body_score'] = result.apply(f, axis=1)
result

,level_1,liststring,Shape,times_in_lines,when_body_score
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,3.15pm,0.5
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,3.15pm,0.5


In [106]:
#df_calc = result[['locs_in_lines', 'pos_score']]
result["times"]=result["times_in_lines"].str.split(", ")
result = result.explode("times")

#df_calc = df_calc.groupby('splited')['C'].sum().reset_index() 
print(result)

   level_1                                         liststring  \
1        1  One person has died after two motorbikes crash...   
5        1  One person has died and another has minor inju...   

                                               Shape times_in_lines  \
1  The crash occurred at 3.15pm on Bell Rd in Nuk...         3.15pm   
5  The crash involved two motorcycles and was rep...         3.15pm   

   when_body_score   times  
1              0.5  3.15pm  
5              0.5  3.15pm  


In [107]:
df_when_bodytext_score = result.groupby('times')['when_body_score'].sum().reset_index() 
df_when_bodytext_score

,times,when_body_score
0,3.15pm,1.0


In [108]:
if not titletext.empty:
    df_when_title_score = titletext[['times_in_title', 'when_title_score']]
    df_when_title_score['times'] = df_when_title_score['times_in_title']

In [109]:
if not titletext.empty:
    df_when_position_score = pd.merge(df_when_bodytext_score, df_when_title_score, on=['times'],how='outer')
else:
    df_when_position_score = df_when_bodytext_score

In [110]:
if not titletext.empty:
    df_when_position_score['full_position_score_when'] = df_when_position_score['when_body_score'] + df_when_position_score['when_title_score']
else:
    df_when_position_score['full_position_score_when'] = df_when_bodytext_score['when_body_score']

In [111]:
df_when_position_score

,times,when_body_score,full_position_score_when
0,3.15pm,1.0,1.0


In [112]:
df_when_frequency

,times,frequency
0,3.15pm,2


In [113]:
df_position_and_frequency_when = pd.merge(df_when_frequency, df_when_position_score, on=['times'],how='outer')
df_position_and_frequency_when

,times,frequency,when_body_score,full_position_score_when
0,3.15pm,2,1.0,1.0


In [114]:
df_position_and_frequency_when['position_and_frequency_score'] = df_position_and_frequency_when['full_position_score_when'] + df_position_and_frequency_when['frequency']
df_position_and_frequency_when

,times,frequency,when_body_score,full_position_score_when,position_and_frequency_score
0,3.15pm,2,1.0,1.0,3.0


In [115]:
#when = max(counts_when.items(), key=operator.itemgetter(1))[0]
when = df_position_and_frequency_when.loc[df_position_and_frequency_when['position_and_frequency_score'].idxmax()].values[0]

print("=============================")
print("When? : ", when)
print("=============================")

When? :  3.15pm


In [116]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df_position_and_frequency_when['locnormalized'] = scaler.fit_transform(df_position_and_frequency_when[['position_and_frequency_score']])
df_position_and_frequency_when[['times','locnormalized']]

,times,locnormalized
0,3.15pm,0.0


### what?

In [117]:
What_Title = bigdata['What_Title'].tolist()
What_Body = bigdata['What_Body'].tolist()

What_Title, What_Body

(['dead', 'leaves one dead'],
 ['died, occurred, suffered minor injuries',
  'died, involved two motorcycles, 15pm, attending the scene, attending the scene'])

In [118]:
All_What = What_Title + What_Body
All_What

# words = [w.replace('the', '') for w in WT]
# words = [w.replace('Eastern Bay', 'Eastern Bay of Plenty') for w in words]
# words = [w.replace('Eastern', '') for w in words]

All_What_List = list(filter(None, All_What))
All_What_List

['dead',
 'leaves one dead',
 'died, occurred, suffered minor injuries',
 'died, involved two motorcycles, 15pm, attending the scene, attending the scene']

In [119]:
str_list3 = ",".join(All_What_List).split(',')
str_list3 = [x.strip(' ') for x in str_list3] #Remove space before ' Bay of Plenty
str_list3

['dead',
 'leaves one dead',
 'died',
 'occurred',
 'suffered minor injuries',
 'died',
 'involved two motorcycles',
 '15pm',
 'attending the scene',
 'attending the scene']

In [120]:
counts = Counter(str_list3)
counts_what = dict(counts)
print(counts_what)

{'dead': 1, 'leaves one dead': 1, 'died': 2, 'occurred': 1, 'suffered minor injuries': 1, 'involved two motorcycles': 1, '15pm': 1, 'attending the scene': 2}


In [121]:
df_phrases = pd.DataFrame(list(counts_what.items()), columns=['phrases', 'frequency'])
df_phrases #Frequency dataframe-What

,phrases,frequency
0,dead,1
1,leaves one dead,1
2,died,2
3,occurred,1
4,suffered minor injuries,1
5,involved two motorcycles,1
6,15pm,1
7,attending the scene,2


In [124]:
phrase_list = df_phrases.phrases.tolist()
phrase_list

['dead',
 'leaves one dead',
 'died',
 'occurred',
 'suffered minor injuries',
 'involved two motorcycles',
 '15pm',
 'attending the scene']

#### Position score

In [125]:
titletext = bigdata[["Title"]]
titletext

,Title
0,One dead after two motorbikes crash in Bay of ...
1,Motorcycles crash leaves one dead near Whakatāne


In [126]:
titletext['phrase_in_title'] = titletext.Title.str.findall(f'({"|".join(phrase_list)})', flags=re.IGNORECASE)
titletext

,Title,phrase_in_title
0,One dead after two motorbikes crash in Bay of ...,[dead]
1,Motorcycles crash leaves one dead near Whakatāne,[leaves one dead]


In [127]:
titletext['phrase_in_title'] = titletext.phrase_in_title.astype(str).str.replace('\[|\]|\'', '')
titletext

,Title,phrase_in_title
0,One dead after two motorbikes crash in Bay of ...,dead
1,Motorcycles crash leaves one dead near Whakatāne,leaves one dead


In [128]:
titletext['phrase_in_title'].replace('', np.nan, inplace=True)
titletext

,Title,phrase_in_title
0,One dead after two motorbikes crash in Bay of ...,dead
1,Motorcycles crash leaves one dead near Whakatāne,leaves one dead


In [129]:
titletext = titletext.dropna(subset=['phrase_in_title'])
titletext

,Title,phrase_in_title
0,One dead after two motorbikes crash in Bay of ...,dead
1,Motorcycles crash leaves one dead near Whakatāne,leaves one dead


In [130]:
titletext['phrase_title_score'] = 1.0
titletext

,Title,phrase_in_title,phrase_title_score
0,One dead after two motorbikes crash in Bay of ...,dead,1.0
1,Motorcycles crash leaves one dead near Whakatāne,leaves one dead,1.0


In [131]:
bodytext

,liststring,Shape
0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
4,One person has died and another has minor inju...,One person has died and another has minor inju...
5,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
6,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [132]:
grouped = bodytext.groupby('liststring', as_index=False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
result = grouped_index.drop('level_0',axis = 1).set_index('level_1')
result

,liststring,Shape
level_1,,
0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
0,One person has died and another has minor inju...,One person has died and another has minor inju...
1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [133]:
result = result.reset_index()
result

,level_1,liststring,Shape
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...


In [134]:
import re
# result['location_mapped_title'] = (result.Shape.str
#                              .findall('|'.join(loc_list))
#                              .str[0])

#x = result.Shape.str.extractall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result['phrase_in_lines'] = result.Shape.str.findall(f'({"|".join(phrase_list)})', flags=re.IGNORECASE)
result

,level_1,liststring,Shape,phrase_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,[died]
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"[occurred, 15pm]"
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,[suffered minor injuries]
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,[]
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,[died]
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"[involved two motorcycles, 15pm]"
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,[attending the scene]


In [135]:
result['phrase_in_lines'] = result.phrase_in_lines.astype(str).str.replace('\[|\]|\'', '')

result

,level_1,liststring,Shape,phrase_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,died
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"occurred, 15pm"
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,suffered minor injuries
3,3,One person has died after two motorbikes crash...,WorkSafe had been advised and the Serious Cras...,
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,died
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"involved two motorcycles, 15pm"
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,attending the scene


In [136]:
result['phrase_in_lines'].replace("", np.nan, inplace=True)
result = result.dropna(subset=['phrase_in_lines'])
result

,level_1,liststring,Shape,phrase_in_lines
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,died
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"occurred, 15pm"
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,suffered minor injuries
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,died
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"involved two motorcycles, 15pm"
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,attending the scene


In [137]:
result['what_body_score'] = result.apply(f, axis=1)
result

,level_1,liststring,Shape,phrase_in_lines,what_body_score
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,died,1.00
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"occurred, 15pm",0.50
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,suffered minor injuries,0.25
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,died,1.00
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"involved two motorcycles, 15pm",0.50
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,attending the scene,0.25


In [138]:
#df_calc = result[['locs_in_lines', 'pos_score']]
result["phrases"]=result["phrase_in_lines"].str.split(", ")
result = result.explode("phrases")

#df_calc = df_calc.groupby('splited')['C'].sum().reset_index() 
result

,level_1,liststring,Shape,phrase_in_lines,what_body_score,phrases
0,0,One person has died after two motorbikes crash...,One person has died after two motorbikes crash...,died,1.00,died
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"occurred, 15pm",0.50,occurred
1,1,One person has died after two motorbikes crash...,The crash occurred at 3.15pm on Bell Rd in Nuk...,"occurred, 15pm",0.50,15pm
2,2,One person has died after two motorbikes crash...,Another person suffered minor injuries in the ...,suffered minor injuries,0.25,suffered minor injuries
4,0,One person has died and another has minor inju...,One person has died and another has minor inju...,died,1.00,died
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"involved two motorcycles, 15pm",0.50,involved two motorcycles
5,1,One person has died and another has minor inju...,The crash involved two motorcycles and was rep...,"involved two motorcycles, 15pm",0.50,15pm
6,2,One person has died and another has minor inju...,The Serious Crash Unit and WorkSafe are attend...,attending the scene,0.25,attending the scene


In [139]:
df_bodytext_score_what = result.groupby('phrases')['what_body_score'].sum().reset_index() 
df_bodytext_score_what

,phrases,what_body_score
0,15pm,1.00
1,attending the scene,0.25
2,died,2.00
3,involved two motorcycles,0.50
4,occurred,0.50
5,suffered minor injuries,0.25


In [140]:
df_title_score_what = titletext[['phrase_in_title', 'phrase_title_score']]
df_title_score_what['phrases'] = df_title_score_what['phrase_in_title']
df_title_score_what

,phrase_in_title,phrase_title_score,phrases
0,dead,1.0,dead
1,leaves one dead,1.0,leaves one dead


In [141]:
df_position_score_what = pd.merge(df_bodytext_score_what, df_title_score_what, on=['phrases'],how='outer')
df_position_score_what

,phrases,what_body_score,phrase_in_title,phrase_title_score
0,15pm,1.00,NaN,NaN
1,attending the scene,0.25,NaN,NaN
2,died,2.00,NaN,NaN
3,involved two motorcycles,0.50,NaN,NaN
4,occurred,0.50,NaN,NaN
5,suffered minor injuries,0.25,NaN,NaN
6,dead,NaN,dead,1.0
7,leaves one dead,NaN,leaves one dead,1.0


In [142]:
df_position_score_what['full_position_score_what'] = df_position_score_what['what_body_score'] + df_position_score_what['phrase_title_score']
df_position_score_what

,phrases,what_body_score,phrase_in_title,phrase_title_score,full_position_score_what
0,15pm,1.00,NaN,NaN,NaN
1,attending the scene,0.25,NaN,NaN,NaN
2,died,2.00,NaN,NaN,NaN
3,involved two motorcycles,0.50,NaN,NaN,NaN
4,occurred,0.50,NaN,NaN,NaN
5,suffered minor injuries,0.25,NaN,NaN,NaN
6,dead,NaN,dead,1.0,NaN
7,leaves one dead,NaN,leaves one dead,1.0,NaN


In [143]:
df_position_score_what.full_position_score_what.fillna(df_position_score_what.what_body_score, inplace=True)
df_position_score_what

,phrases,what_body_score,phrase_in_title,phrase_title_score,full_position_score_what
0,15pm,1.00,NaN,NaN,1.00
1,attending the scene,0.25,NaN,NaN,0.25
2,died,2.00,NaN,NaN,2.00
3,involved two motorcycles,0.50,NaN,NaN,0.50
4,occurred,0.50,NaN,NaN,0.50
5,suffered minor injuries,0.25,NaN,NaN,0.25
6,dead,NaN,dead,1.0,NaN
7,leaves one dead,NaN,leaves one dead,1.0,NaN


In [144]:
df_position_score_what.full_position_score_what.fillna(df_position_score_what.phrase_title_score, inplace=True)
df_position_score_what

,phrases,what_body_score,phrase_in_title,phrase_title_score,full_position_score_what
0,15pm,1.00,NaN,NaN,1.00
1,attending the scene,0.25,NaN,NaN,0.25
2,died,2.00,NaN,NaN,2.00
3,involved two motorcycles,0.50,NaN,NaN,0.50
4,occurred,0.50,NaN,NaN,0.50
5,suffered minor injuries,0.25,NaN,NaN,0.25
6,dead,NaN,dead,1.0,1.00
7,leaves one dead,NaN,leaves one dead,1.0,1.00


In [145]:
df_phrases #Frequency dataframe-What

,phrases,frequency
0,dead,1
1,leaves one dead,1
2,died,2
3,occurred,1
4,suffered minor injuries,1
5,involved two motorcycles,1
6,15pm,1
7,attending the scene,2


In [146]:
df_position_and_frequency_what = pd.merge(df_phrases, df_position_score_what, on=['phrases'],how='outer')
df_position_and_frequency_what

,phrases,frequency,what_body_score,phrase_in_title,phrase_title_score,full_position_score_what
0,dead,1,NaN,dead,1.0,1.00
1,leaves one dead,1,NaN,leaves one dead,1.0,1.00
2,died,2,2.00,NaN,NaN,2.00
3,occurred,1,0.50,NaN,NaN,0.50
4,suffered minor injuries,1,0.25,NaN,NaN,0.25
5,involved two motorcycles,1,0.50,NaN,NaN,0.50
6,15pm,1,1.00,NaN,NaN,1.00
7,attending the scene,2,0.25,NaN,NaN,0.25


In [147]:
df_position_and_frequency_what['position_and_frequency_score_what'] = df_position_and_frequency_what['full_position_score_what'] + df_position_and_frequency_what['frequency']
df_position_and_frequency_what

,phrases,frequency,what_body_score,phrase_in_title,phrase_title_score,full_position_score_what,position_and_frequency_score_what
0,dead,1,NaN,dead,1.0,1.00,2.00
1,leaves one dead,1,NaN,leaves one dead,1.0,1.00,2.00
2,died,2,2.00,NaN,NaN,2.00,4.00
3,occurred,1,0.50,NaN,NaN,0.50,1.50
4,suffered minor injuries,1,0.25,NaN,NaN,0.25,1.25
5,involved two motorcycles,1,0.50,NaN,NaN,0.50,1.50
6,15pm,1,1.00,NaN,NaN,1.00,2.00
7,attending the scene,2,0.25,NaN,NaN,0.25,2.25


In [148]:
what = df_position_and_frequency_what.loc[df_position_and_frequency_what['position_and_frequency_score_what'].idxmax()].values[0]

print("=============================")
print("What? : ", what)
print("=============================")

What? :  died


In [149]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df_position_and_frequency_what['locnormalized'] = scaler.fit_transform(df_position_and_frequency_what[['position_and_frequency_score_what']])
df_position_and_frequency_what[['phrases','locnormalized']]

,phrases,locnormalized
0,dead,0.272727
1,leaves one dead,0.272727
2,died,1.000000
3,occurred,0.090909
4,suffered minor injuries,0.000000
5,involved two motorcycles,0.090909
6,15pm,0.272727
7,attending the scene,0.363636
